In [ ]:
# Import modules
import numpy as np

# Import functions
from local_global_matrices import *
from assembly_K_matrices import *
from RegularSudomainsMesh import RegularSubdomainsMesh
from utils import *

%load_ext autoreload
%autoreload 2

In [ ]:
# Import data
d_dat = np.genfromtxt('data/d.dat')
fP_dat = np.genfromtxt('data/fP.dat')
fr_dat = np.genfromtxt('data/fr.dat')
Ks = np.genfromtxt('data/localK.dat')
solution = np.genfromtxt('data/solution.dat')

In [ ]:
# Initial data
# Number of subdomains
Nsub_x = 4
Nsub_y = 3

# Number of remaining nodes in each subdomain
Nr_x = 4
Nr_y = 3

# Local remaining and primal indices
rs = np.array([1, 2, 4, 5, 6, 7, 9, 10])
qs = np.array([0, 3, 8, 11])

In [ ]:
# Transformation matrices A
# Primal nodes local-global transformation matrices
APq = create_APq_matrices(Nsub_x, Nsub_y)

# Remaining nodes local-global transformation matrices
ARr = create_ARr_matrices(Nsub_x, Nsub_y, Nr_x, Nr_y)

In [ ]:
# Stiffness matrices K
KRR = assembly_KRR_matrix(Ks, ARr, rs)
KPP = assembly_KPP_matrix(Ks, APq, qs)
KRP = assembly_KPR_matrix(Ks, APq, ARr, qs, rs)
KPR = KRP.T

In [ ]:
Nr = Nr_x*Nr_y - 4  # Number of local remaining nodes r
NR = Nr*(Nsub_x*Nsub_y)  # Number of total remaining nodes R
Nq = 4 # Number of primal nodes in each subdomain 
NP = (Nsub_x + 1)*(Nsub_y + 1) # Total number of primal nodes

In [ ]:
NlambdaR = (Nsub_x*(Nr_x - 2)) * (Nsub_y - 1) + (Nsub_y*(Nr_y - 2)) * (Nsub_x - 1)
NlambdaP_Dir = 4 # hardcoded
NlambdaR_Dir = 3 # hardcoded



Nlambda = NlambdaR + NlambdaR_Dir + NlambdaP_Dir

In [ ]:
d = np.zeros(Nlambda)
np.shape(d)

In [ ]:
Nr = Nr_x*Nr_y - 4  # Number of local remaining nodes r
NR = Nr*(Nsub_x*Nsub_y)  # Number of total remaining nodes R
BlambdaR = np.zeros([Nlambda, NR])

for j in range(Nsub_y):
    for i in range(Nsub_x):
        Brs = np.zeros([Nlambda, Nr])
        # Local rs nodes in boundaries
        rs_bot = np.arange(Nr_x - 2)
        rs_top = np.arange(Nr_x - 2) + (Nr_y - 2)*Nr_x + (Nr_x - 2)
        rs_left = np.arange(Nr_x - 2, (Nr_y - 2)*(Nr_x) + Nr_x - 2, Nr_x)
        rs_right = rs_left + Nr_x - 1

        Nrs_hor = Nr_x - 2
        Nrs_ver = Nr_y - 2

        # left
        if i > 0:
            lambda_left = np.arange((i-1) + j*(Nr_y - 2)*(Nsub_x - 1), Nrs_ver*(Nsub_x - 1) + (i-1) + j*(Nr_y - 2)*(Nsub_x - 1),step=Nsub_x - 1)
            # print('left', lambda_left)
            for lambda_, rs in zip(lambda_left, rs_left):
                Brs[int(lambda_), int(rs)] = -1
        # right
        if i < Nsub_x - 1:
            lambda_right = np.arange((i) + j*(Nr_y - 2)*(Nsub_x - 1), Nrs_ver*(Nsub_x - 1) + (i) + j*(Nr_y - 2)*(Nsub_x - 1),step=Nsub_x - 1)
            # print('right', lambda_right)
            for lambda_, rs in zip(lambda_right, rs_right):
                Brs[int(lambda_), int(rs)] = 1
        # bottom
        NlambdaR_hor = (Nr_y - 2)*(Nsub_y)*(Nsub_x - 1)
        if j > 0:
            lambda_bot = np.arange(NlambdaR_hor + (j - 1)*(Nr_x - 2)*Nsub_x + i*(Nr_x - 2), NlambdaR_hor + (j - 1)*(Nr_x - 2)*Nsub_x + i*(Nr_x - 2) + (Nr_x - 2))
            # print('bot', lambda_bot)
            for lambda_, rs in zip(lambda_bot, rs_bot):
                Brs[int(lambda_), int(rs)] = 1
        # top
        if j < Nsub_y - 1:
            lambda_top = np.arange(NlambdaR_hor + (j)*(Nr_x - 2)*Nsub_x + i*(Nr_x - 2), NlambdaR_hor + (j)*(Nr_x - 2)*Nsub_x + i*(Nr_x - 2) + (Nr_x - 2))
            # print('top', lambda_top)
            for lambda_, rs in zip(lambda_top, rs_top):
                Brs[int(lambda_), int(rs)] = -1

        BlambdaR += Brs @ ARr[i + j*Nsub_x].T

plot_sparse_matrix(BlambdaR)

In [ ]:
# Dirichlet boundary conditions
# Left wall remaining

Nr_x = 4
Nr_y = 3
Nsub_x = 4
Nsub_y = 3
NlambdaR = 25
BlambdaR_aux = np.zeros([Nlambda, NR])
for j in range(Nsub_y):
    Brs = np.zeros([Nlambda, Nr])
    rs_left = np.arange(Nr_x - 2, (Nr_y - 2)*(Nr_x) + Nr_x - 2, Nr_x)
    lambda_left = np.arange(NlambdaR + (Nr_y - 1)*j + 1, NlambdaR + (Nr_y - 1)*j + 1 + Nr_y - 2)
    #d_left = np.arange(1 + j*(Nr_y - 1), 1+j*(Nr_y - 1) + Nr_y - 2)
    for rs, lambda_ in zip(rs_left, lambda_left):
        Brs[lambda_, rs] = 1
    BlambdaR_aux += Brs @ ARr[j*Nsub_x].T
    
BlambdaR += BlambdaR_aux    



In [ ]:
#Left wall primal
BlambdaP = np.zeros([Nlambda, NP])

for j in range(Nsub_y):
    Bqs = np.zeros([Nlambda, Nq])
    if j == 0:
        Bqs[NlambdaR, 0] = 1
    Bqs[NlambdaR + j*(Nr_y - 1) + Nr_y - 1, 2] = 1
    BlambdaP += Bqs @ APq[j*Nsub_x].T
    
plot_sparse_matrix(KRP)

In [ ]:
# Assembly d
d[NlambdaR:]=d_dat

In [ ]:
# Assembly f
# fP
fP = fP_dat

# fR
fR = np.zeros(NR)

In [ ]:
NR

In [ ]:
for j in range(Nsub_y):
    for i in range(Nsub_x):
        Rs = []
        for sj in range(Nr_y):
            if sj == 0:
                Rs = np.concatenate([Rs, np.arange(j*(Nsub_x*Nr) + i*(Nr_x - 2), j*(Nsub_x*Nr) + i*(Nr_x - 2) + Nr_x - 2)])
            elif sj == Nr_y - 1:
                Rs = np.concatenate([Rs, np.arange(j*(Nsub_x*Nr) + i*(Nr_x - 2) + Nsub_x*Nr_x*(Nr_y - 2) + Nsub_x*(Nr_x - 2),
                                                   j*(Nsub_x*Nr) + i*(Nr_x - 2) + Nsub_x*Nr_x*(Nr_y - 2) + Nsub_x*(Nr_x - 2) + Nr_x - 2)])
            else:
                Rs = np.concatenate([Rs, np.arange(j*(Nsub_x*Nr) + i*(Nr_x) + Nsub_x*Nr_x*(sj - 1) + Nsub_x*(Nr_x - 2), 
                                                   j*(Nsub_x*Nr) + i*(Nr_x) + Nsub_x*Nr_x*(sj - 1) + Nsub_x*(Nr_x - 2) + Nr_x)])
        fR[Rs.astype(int)] = fr_dat



In [ ]:
print(np.shape(KPP), np.shape(KPR), np.shape(np.linalg.inv(KRR)), np.shape(KRP))

In [ ]:
KRR_inv = np.linalg.inv(KRR)
SPP = KPP - KPR @ KRR_inv @ KRP
SPP_inv = np.linalg.inv(SPP)
np.shape(KRR_inv)

In [ ]:
fPH = fP - KPR @ KRR_inv @ fR

In [ ]:
dH = -d + BlambdaP @ SPP_inv @ fPH + BlambdaR @ KRR_inv @ fR - BlambdaR @ KRR_inv @ KRP @ SPP_inv @ fPH
np.shape(dH)

In [ ]:
F = -BlambdaP @ SPP_inv @ KPR @ KRR_inv @ BlambdaR.T + BlambdaP @ SPP_inv @ BlambdaP.T + BlambdaR @ KRR_inv @ KRP @ SPP_inv @ KPR @ KRR_inv @ BlambdaR.T - BlambdaR @ KRR_inv @ KRP @ SPP_inv @ BlambdaP.T + BlambdaR @ KRR_inv @ BlambdaR.T
np.shape(F)
plot_sparse_matrix(F)

In [ ]:
lambda_ = np.linalg.solve(F, dH)

In [ ]:
lambda_

In [ ]:
uP = SPP_inv @ fPH + SPP_inv @ KPR @ KRR_inv @ BlambdaR.T @ lambda_ - SPP_inv @ BlambdaP.T @ lambda_
uP

In [ ]:
uR = KRR_inv @ (fR - KRP @ uP - BlambdaR.T @ lambda_)
uR

In [ ]:
     
    
   

mesh = RegularSubdomainsMesh(4, 3, 4, 3)

